<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="https://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/llibre-estil/logo-UOC-masterbrand.jpg" align="left">
</div>
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">TFM - UEBA: Detección de Anomalías</p>
<p style="margin: 0; text-align:right;">20231 - Máster universitario en Ciencias de datos (<i>Data science</i>)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicaciones</p>
</div>
</div>
<div style="width: 100%; clear: both;">
<div style="width:100%;">&nbsp;</div>

<div class="alert alert-block alert-info">
    <strong>Nombre y apellidos:</strong> Alejandro Ortega de los Ríos
</div>

# EXTRACCIÓN DE CARACTERÍSTICAS - LDAP

In [34]:
import pandas as pd
import numpy as np

In [38]:
ldap_df = pd.read_csv('../data/ldap.csv')
features = pd.read_csv('features.csv')

In [6]:
ldap_df.head()

,employee_name,user_id,email,role,business_unit,functional_unit,department,team,supervisor
0,Calvin Edan Love,CEL0561,Calvin.Edan.Love@dtaa.com,ComputerProgrammer,1,2 - ResearchAndEngineering,2 - SoftwareManagement,3 - Software,Stephanie Briar Harrington
1,Christine Reagan Deleon,CRD0624,Christine.Reagan.Deleon@dtaa.com,Salesman,1,5 - SalesAndMarketing,2 - Sales,3 - RegionalSales,Winter Veda Burks
2,Jade Felicia Caldwell,JFC0557,Jade.Felicia.Caldwell@dtaa.com,SoftwareEngineer,1,2 - ResearchAndEngineering,2 - SoftwareManagement,3 - Software,Stephanie Briar Harrington
3,Aquila Stewart Dejesus,ASD0577,Aquila.Stewart.Dejesus@dtaa.com,ProductionLineWorker,1,3 - Manufacturing,3 - Assembly,3 - AssemblyDept,Whilemina Pandora England
4,Micah Abdul Rojas,MAR0955,Micah.Abdul.Rojas@dtaa.com,ProductionLineWorker,1,3 - Manufacturing,3 - Assembly,6 - AssemblyDept,Sandra Beverly Diaz


## 1. Puesto de trabajo

In [13]:
#Compruebo número de categorías por variable
roles = ldap_df['role'].unique().tolist()
functional_units = ldap_df['functional_unit'].unique().tolist()
departments = ldap_df['department'].unique().tolist()
teams = ldap_df['team'].unique().tolist()

print("Number of Roles: ", len(roles))
print("Number of Functional Units: ", len(functional_units))
print("Number of Departments: ", len(departments))
print("Number of Teams: ", len(teams))

Number of Roles:  42
Number of Functional Units:  7
Number of Departments:  23
Number of Teams:  39


In [35]:
#Codifico los puestos de trabajo
regex_num = r'^(\d+)'
ldap_df['team'] = ldap_df['team'].str.extract(regex_num)
ldap_df['functional_unit'] = ldap_df['functional_unit'].str.extract(regex_num)
ldap_df['department'] = ldap_df['department'].str.extract(regex_num)
ldap_df['role'], _ = pd.factorize(ldap_df['role'])

In [59]:
#Compruebo si existen valores nulos
print("Team Nan values:", any(ldap_df['team'].isna()))
print("Functional Unit Nan values:", any(ldap_df['functional_unit'].isna()))
print("Department Nan values:", any(ldap_df['department'].isna()))
print("Role Nan values:", any(ldap_df['role'].isna()))

Team Nan values: True
Functional Unit Nan values: True
Department Nan values: True
Role Nan values: False


In [61]:
#Transformo Nan a -1 (Corresponde con jefes de departamento)
ldap_df['team'] = ldap_df['team'].fillna(-1)
ldap_df['functional_unit'] = ldap_df['functional_unit'].fillna(-1)
ldap_df['department'] = ldap_df['department'].fillna(-1)
ldap_df['role'] = ldap_df['role'].fillna(-1)

In [62]:
ldap_df.head()

,employee_name,user_id,email,role,business_unit,functional_unit,department,team,supervisor
0,Calvin Edan Love,CEL0561,Calvin.Edan.Love@dtaa.com,0,1,2,2,3,Stephanie Briar Harrington
1,Christine Reagan Deleon,CRD0624,Christine.Reagan.Deleon@dtaa.com,1,1,5,2,3,Winter Veda Burks
2,Jade Felicia Caldwell,JFC0557,Jade.Felicia.Caldwell@dtaa.com,2,1,2,2,3,Stephanie Briar Harrington
3,Aquila Stewart Dejesus,ASD0577,Aquila.Stewart.Dejesus@dtaa.com,3,1,3,3,3,Whilemina Pandora England
4,Micah Abdul Rojas,MAR0955,Micah.Abdul.Rojas@dtaa.com,3,1,3,3,6,Sandra Beverly Diaz


## 2. Espacio de Características

In [65]:
#Me quedo con un unico registro por usuario
ldap_df_cleaned = ldap_df.drop_duplicates(subset=['user_id'])[['user_id', 'role', 'functional_unit', 'department', 'team']]
ldap_df_cleaned = ldap_df_cleaned.rename(columns={'user_id':'user'})

In [66]:
#Combino con el dataset features
features_merged = features.merge(ldap_df_cleaned, on='user', how='left')
features_merged.drop(columns=['Unnamed: 0'], inplace=True)

In [69]:
features_merged.head()

,user,email_size,attachments,recipients,working_day_emails,address_count,weekend_emails,insider,working_day_files,weekend_files,...,tld_net,tld_org,tld_ph,tld_tv,tld_uk,tld_us,role,functional_unit,department,team
0,LAP0338,30020.394184,0.377839,1.638930,13.655072,2,2,0,0,0,...,2.390805,1.505495,1.0,0.0,1.000000,0.0,6,5,3,1
1,MOH0273,30397.402083,0.758333,1.670833,2.962963,2,8,0,23,22,...,6.741848,2.861386,0.0,0.0,1.000000,0.0,29,2,3,4
2,HVB0037,29958.497676,0.000000,1.658367,8.705202,3,13,0,0,0,...,3.812317,3.802260,1.0,0.0,1.654321,0.0,25,1,6,1
3,NWK0215,29828.181548,0.431548,1.651786,1.000000,2,12,0,0,0,...,2.440758,4.394299,0.0,1.0,2.354167,0.0,25,1,6,1
4,LRR0148,29895.532625,0.930197,1.587253,2.941964,2,13,0,3,3,...,5.390052,17.402010,1.0,0.0,3.333333,1.0,14,4,3,-1


In [70]:
#Guardo espacio de características en fichero
features_merged.to_csv('features.csv')